In [2]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Koneksi ke MongoDB
try:
    client = MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=3000)
    client.server_info()  # Memicu exception jika tidak bisa konek
    print("✅ Berhasil terhubung ke MongoDB.")
except Exception as e:
    print("❌ Gagal terhubung ke MongoDB:", e)


db = client["workout_db"]  # Ganti sesuai nama database
collection = db["obesity"]  # Ganti sesuai nama koleksi (tabel MongoDB)

✅ Berhasil terhubung ke MongoDB.


In [4]:
data = pd.DataFrame(list(collection.find()))

# Pastikan data tidak kosong
if data.empty:
    raise ValueError("Data dari MongoDB kosong. Pastikan koleksi 'obesity' terisi!")
else:
    print("✅ Data berhasil diambil dari MongoDB. Contoh data:")
    print(data.head())
    print(f"Jumlah baris: {data.shape[0]}, Jumlah kolom: {data.shape[1]}")

✅ Data berhasil diambil dari MongoDB. Contoh data:
                        _id  Age  Gender      Height     Weight        BMI  \
0  680a16870893fe10f40820d4   56    Male  173.575262  71.982051  23.891783   
1  680a16870893fe10f40820d5   69    Male  164.127306  89.959256  33.395209   
2  680a16870893fe10f40820d6   46  Female  168.072202  72.930629  25.817737   
3  680a16870893fe10f40820d7   32    Male  168.459633  84.886912  29.912247   
4  680a16870893fe10f40820d8   60    Male  183.568568  69.038945  20.487903   

   PhysicalActivityLevel ObesityCategory  
0                      4   Normal weight  
1                      2           Obese  
2                      4      Overweight  
3                      3      Overweight  
4                      3   Normal weight  
Jumlah baris: 1000, Jumlah kolom: 8


In [5]:
# Cleansing data: hapus _id dan kolom tak relevan, serta drop NaN pada ObesityCategory
data = data.drop(columns=[col for col in ['_id', 'PhysicalActivityLevel'] if col in data.columns])
data = data.dropna(subset=['ObesityCategory']) if 'ObesityCategory' in data.columns else data

# Tampilkan hasil setelah cleansing
print("✅ Data setelah proses cleansing:")
print(data.head())  # Tampilkan 5 baris pertama
print(f"Jumlah baris: {data.shape[0]}, Jumlah kolom: {data.shape[1]}")

✅ Data setelah proses cleansing:
   Age  Gender      Height     Weight        BMI ObesityCategory
0   56    Male  173.575262  71.982051  23.891783   Normal weight
1   69    Male  164.127306  89.959256  33.395209           Obese
2   46  Female  168.072202  72.930629  25.817737      Overweight
3   32    Male  168.459633  84.886912  29.912247      Overweight
4   60    Male  183.568568  69.038945  20.487903   Normal weight
Jumlah baris: 1000, Jumlah kolom: 6


In [9]:
# Encode ObesityCategory
le = LabelEncoder()
data['ObesityCategory'] = le.fit_transform(data['ObesityCategory'])
joblib.dump(le, 'label_encoder_ObesityCategory.pkl')
print(data['ObesityCategory'])

0      0
1      1
2      2
3      2
4      0
      ..
995    2
996    1
997    1
998    0
999    0
Name: ObesityCategory, Length: 1000, dtype: int64


In [11]:
# Pisahkan fitur dan label
X = data.drop(columns=['ObesityCategory'])
y = data['ObesityCategory']

In [12]:
# Fitur kategorikal dan numerik
categorical_features = ['Gender']
numerical_features = ['Age', 'Height', 'Weight', 'BMI']

# One-hot encoding untuk Gender
encoder = OneHotEncoder(handle_unknown='ignore')
categorical_encoded = encoder.fit_transform(X[categorical_features]).toarray()
joblib.dump(encoder, 'encoder.pkl')
print(categorical_encoded)

[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [15]:
# Gabungkan kembali dengan fitur numerik
X_final = pd.concat([
    pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out()),
    X[numerical_features].reset_index(drop=True)
], axis=1)

# Tampilkan hasil gabungan fitur
print("✅ Fitur akhir setelah penggabungan kategorikal dan numerik:")
print(X_final.head())  # Tampilkan 5 baris pertama
print(f"Jumlah baris: {X_final.shape[0]}, Jumlah kolom: {X_final.shape[1]}")

✅ Fitur akhir setelah penggabungan kategorikal dan numerik:
   Gender_Female  Gender_Male  Age      Height     Weight        BMI
0            0.0          1.0   56  173.575262  71.982051  23.891783
1            0.0          1.0   69  164.127306  89.959256  33.395209
2            1.0          0.0   46  168.072202  72.930629  25.817737
3            0.0          1.0   32  168.459633  84.886912  29.912247
4            0.0          1.0   60  183.568568  69.038945  20.487903
Jumlah baris: 1000, Jumlah kolom: 6


In [16]:
# Scaling fitur numerik
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_final)
joblib.dump(scaler, 'scaler.pkl')
print(X_scaled)

[[-0.95501094  0.95501094  0.33929464  0.34186404  0.05007594 -0.16096979]
 [-0.95501094  0.95501094  1.05732009 -0.57498474  1.20973904  1.37411521]
 [ 1.04710843 -1.04710843 -0.21303263 -0.19216404  0.11126628  0.15012898]
 ...
 [ 1.04710843 -1.04710843 -0.04733445 -1.30826818  0.49016063  1.17233664]
 [-0.95501094  0.95501094  0.78115646 -0.56868513 -0.85328182 -0.54635003]
 [ 1.04710843 -1.04710843  0.89162191  0.82337373  0.24231534 -0.22148122]]


In [17]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Training model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
joblib.dump(model, 'bmi_classification_model_rf.pkl')


['bmi_classification_model_rf.pkl']

In [13]:
# Evaluasi
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print(f"Akurasi: {accuracy * 100:.2f}%")
print("\nLaporan Klasifikasi:")
print(report)

Akurasi: 99.50%

Laporan Klasifikasi:
               precision    recall  f1-score   support

Normal weight       1.00      1.00      1.00        72
        Obese       1.00      0.97      0.98        32
   Overweight       0.98      1.00      0.99        65
  Underweight       1.00      1.00      1.00        31

     accuracy                           0.99       200
    macro avg       1.00      0.99      0.99       200
 weighted avg       1.00      0.99      0.99       200

